# Proyecto I: Implementacion de un sistema de Reconocimiento Automático de Habla.

El objetivo de este proyecto es utilizar las herramientas vistas en clase para la implementación de un sistema de reconocimiento automático de habla (ASR-Automatic Speech Recognition). Para lograr este objetivo se realizarán dos implementaciones. La primera de ellas utilizará HMM, y est definida en el **Taller II: Implementing a simple ASR system using HMM**. La segunda Implementación se realizará utilizando redes Neuronales Reurrentes (RNN).

Par al aimplementación del sistema utilizando RNNs, se utilizara la libreria [TensorFlow](https://pypi.org/project/tensorflow/) de Python, especificamente las funciones para la creación de redes neuronales en [keras](https://www.tensorflow.org/guide/keras/sequential_model) y en particular las relacionadas con las redes neuronales recurrentes [(RNN)](https://www.tensorflow.org/guide/keras/rnn). Pueden utilizar las RNN simples, LTSM o las GRU. Sin emabrgo, las que se estudiaron en clase hasta el momento son las RNN simples. Por otro lado no es necesario que usen las RNNs bidirecionales, o las funciones optimizadas par aGPUs.

Para este taller deben seguir los siguientes pasos:

1. Cree una base de datos de entrenamiento, utilizando la segmentación de las palabras en fonemas y los espectrogramas de la señal de voz calculados en el Taller II.
2. Divida los datos entre datos de entrenamiento y validación. Esto lo puede realizar por medio del uso de  la función [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de SciKit-Learn.
3. Construya un aarquitectura para el ASR usando redes neuronales.
4. Evalue el comportamiento del modelo.
5. Ajuste el modelo si lo considera adecuado.
6. Pruebe con los datos de entrenamiento si el modelo produce la secuencia de fonemas indicada.
7. Con el conjunto de palabras de prueba, generado en el taller II, trate de predecir la palbara escrita utilizando el modelo implementado.
8. Discuta sobre el comportamiento del sistema ASR utilziando HMM y RNN. La discusión debe contener al menos la respuest aa las siguientes pregutnas:
    * ¿Cúal modelo e smás facil de entender?
    * ¿Qué modelo funciona mejor? ¿Cúal es la razón para esto?
    * ¿Discuta sobre las ventajas y desventajas del modelo basado en HMM?
    * ¿Discuta sobre las ventajas y desventajas del modelo basado en RNN?
    * ¿Cómo se podria mejorar el sistema desarrollado? ¿Qué hace falta en este sistema ASR?
    * ¿Obtuvó los resultados esperados?
    
Al enviar el proyecto debe incluir los siguientes items:
1. Notebook de Jupyter explicando el desarrollo del proyecto, y con la respuesta a las preguntas realiadas.
2. Archivos de soporte utilizadso, funciones, etc..
3. Grabaciones de las señales de voz utilizadas para entrenar el sistema.
4. Grabaciones de las señales de voz utilizadas para probar el sistema.

**Nota I:** Una guía rápida par ala implementación del modelo de red neuronal utilizando TensorFlow y Python lo pueden encontrar en este [link](https://www.youtube.com/watch?v=BSpXCRTOLJA).

**Nota II:** Recuerde que este proyecto se realiza en grupos de máximo dos personas. También tenga en cuenta que debido a que el taller II hace parte de la evaluación, deben hacerse con el mismo compañero con el que trabajarón ese taller.

**Nota III:** El deadline par ala entrega del proyecto es el **Domingo 28 de Febrero del 2021**.

**Mucha Suerte!!**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
import scipy as sc
from scipy import signal
from scipy.fft import fftshift

import pandas as pd

import librosa
import librosa.display

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import soundfile as sf
import os

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [2]:
fs = 8000 # Numero de muestras por segundo
nBits = 16 # Numero de bits por muestra del audio
ID = -1
seconds = 5 # Duracion de la grabacion
Nfft = 512
fm = np.arange(0, Nfft / 4) * fs / Nfft

In [3]:
nombres = os.listdir('./utils/sounds/')
nombres

['ahijado.wav',
 'balocesto.wav',
 'espantapajaro.wav',
 'jamon.wav',
 'kiosko.wav',
 'llorar.wav',
 'muchacho.wav',
 'murcielago.wav',
 'sound1.wav',
 'sound10.wav',
 'sound2.wav',
 'sound3.wav',
 'sound4.wav',
 'sound5.wav',
 'sound6.wav',
 'sound7.wav',
 'sound8.wav',
 'sound9.wav',
 'terremoto.wav',
 'zapato.wav']

In [4]:
cantidad_fonemas = [7,10,13,5,6,5,6,10,4,8,5,5,5,8,6,7,6,5,8,6]
dicc = {}
for idx,i in enumerate(nombres):
    dicc[i] = cantidad_fonemas[idx]
dicc

{'ahijado.wav': 7,
 'balocesto.wav': 10,
 'espantapajaro.wav': 13,
 'jamon.wav': 5,
 'kiosko.wav': 6,
 'llorar.wav': 5,
 'muchacho.wav': 6,
 'murcielago.wav': 10,
 'sound1.wav': 4,
 'sound10.wav': 8,
 'sound2.wav': 5,
 'sound3.wav': 5,
 'sound4.wav': 5,
 'sound5.wav': 8,
 'sound6.wav': 6,
 'sound7.wav': 7,
 'sound8.wav': 6,
 'sound9.wav': 5,
 'terremoto.wav': 8,
 'zapato.wav': 6}

In [5]:
def getAudios(nombres):
    audios = []
    for i in nombres:
        y_aux, fs_aux = sf.read('./utils/sounds/' + i)
        if len(y_aux.shape) > 1:
            y_aux = y_aux[:,0]
            y = ((1/np.std(y_aux))*y_aux).reshape(len(y_aux))
        else:
            y = ((1/np.std(y_aux))*y_aux).reshape(len(y_aux))
        audios.append(y)
    return audios

In [6]:
def getMelSpectogram(audios,fs,Nfft):
    spectograms = []
    for i in audios:
        Sm = librosa.feature.melspectrogram(y=i, sr=fs, n_fft=Nfft, n_mels = 39)
        spectograms.append(Sm)
    return spectograms

In [7]:
audios = getAudios(nombres)

In [8]:
Spectograms = getMelSpectogram(audios,fs,Nfft)

In [9]:
palabras_codificadas = [[1,9,10,11,1,5,18], [2,1,13,18,16,3,6,22,23,18],[6,22,19,1,16,23,1,19,1,11,1,20,18],
                        [11,1,15,18,16],[12,10,18,22,12,18], [14,18,20,1,20],[15,24,4,1,4,18],[15,24,20,3,10,6,13,1,8,18],
                        [7,6,4,1], [2,10,14,6,23,6,20,1],[13,10,2,20,18],[13,1,19,10,3],[12,18,21,6,20],[23,6,13,6,7,18,16,18],
                        [19,24,6,20,23,1],[19,1,16,23,1,14,1],[11,10,15,6,16,1],[21,6,8,13,1],[23,6,21,6,15,18,23,18],[3,1,19,1,23,18]]
for p in palabras_codificadas:
    p.insert(0,0)
    p.append(0)

In [10]:
dicc_countFon = {}
for i in range(0,27):
    ocurrencia = 0
    for j in palabras_codificadas:
        ocurrencia += j.count(i)
    dicc_countFon[i] = ocurrencia

In [11]:
def getOneCode(palabras_codificadas):
    palabras_onecode = []
    for palabra in palabras_codificadas:
        matriz = np.zeros((24,len(palabra)))
        for i in range(len(palabra)):
            if palabra[i] > 17 and palabra[i] <= 24:
                matriz[palabra[i] - 1,i] = 1
            else:
                matriz[palabra[i],i] = 1
        palabras_onecode.append(matriz)
    return palabras_onecode

In [12]:
matrices_1_0 = getOneCode(palabras_codificadas)

# Pregunta

Cual es la interpretacion de realizar el split de esta forma

In [13]:
X_train, X_test, y_train, y_test = train_test_split(Spectograms, matrices_1_0, test_size=0.3)

# Modelo

In [27]:
X_nuevo = np.zeros((len(X_train),39,240))
for i in range(len(X_train)):
    X_nuevo[i,:,0:X_train[i].shape[1]] = X_train[i]

In [44]:
type(X_nuevo)

numpy.ndarray

In [39]:
X_aux.shape[1:]

TensorShape([39, 240])

In [40]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_aux.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

NotImplementedError: Cannot convert a symbolic Tensor (lstm_10/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
model.fit(x_train,
          y_train,
          epochs=3,
          validation_data=(x_test, y_test))

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

ModuleNotFoundError: No module named 'tensorflow.keras'

In [42]:
mnist = tf.keras.datasets.mnist  # mnist is a dataset of 28x28 images of handwritten digits and their labels
(x_train, y_train),(x_test, y_test) = mnist.load_data()  # unpacks images to x_train/x_test and labels to y_train/y_test

x_train = x_train/255.0
x_test = x_test/255.0

print(x_train.shape)
print(x_train[0].shape)

11493376/11490434 [==============================] - 6s 0us/step
(60000, 28, 28)
(28, 28)


In [43]:
type(x_train)

numpy.ndarray

In [45]:
model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

NotImplementedError: Cannot convert a symbolic Tensor (lstm_11/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported